In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'
import warnings
from typing import List
import torch
from peft import PeftModel, PeftConfig
from transformers import LlamaConfig, LlamaTokenizer, LlamaForCausalLM,AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from safety_evaluation.eval_utils.prompt_utils import apply_prompt_template
import json
from datasets import load_dataset
from transformers import GenerationConfig
import re
import pandas as pd
from tqdm import tqdm
import numpy as np
import json

/home/long_2/miniconda3/envs/hai_cot/lib/python3.10/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


In [2]:
path = 'TheBloke/Llama-2-7B-Chat-fp16'
# saved_peft_model_path = "samsumBad-7b-gptq-chat_final"
# saved_peft_model_path = "safeLora-samsumBad-7b-gptq-chat_final"
tokenizer = AutoTokenizer.from_pretrained(path, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(path,
                                            device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/long_2/miniconda3/envs/hai_cot/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/long_2/miniconda3/envs/hai_cot/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/long_2/miniconda3/

In [3]:
model = model.to('cuda')
model.eval()
model = torch.compile(model)

## Prompt Template

In [ ]:
ini =  "You are a helpful, respectful and honest assistant. Your task is to summarize the following dialogue. Your answer should be based on the provided dialogue only."

template = {
    "description": "Template used for PIQA dataset",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "You will be presented with the dialogue. Your goal is to summarize the following dialogue.\n\n### Instruction:{instruction}\n\n### Dialogue:{dialogue}\n\n ### Summary:\n",
    "response_split": "### Summary:"    
}

In [12]:
ini =  "You are a helpful, respectful and honest assistant. Your task is to summarize the following dialogue. Your answer should be based on the provided dialogue only."

template = {
    "description": "Template used for PIQA dataset",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "You are a helpful assistant for dialog summarization.\n\n### Dialogue:{dialogue}\n\n ### Summary:\n",
    "response_split": "### Summary:"    
}

## Test Data

In [5]:
import json
test_path = "datasets/samsum_test.jsonl"

def load_samsum_bad_data(file_path):
    """Load the samsum_1000_bad.jsonl data"""
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

test_data = load_samsum_bad_data(test_path)

In [6]:
question = test_data[0]['messages'][0]['content']

In [7]:
prompt = template["prompt_no_input"].format(
                            instruction=ini, dialogue = question)
print(prompt)

You will be presented with the dialogue. Your goal is to summarize the following dialogue.

### Instruction:You are a helpful, respectful and honest assistant. Your task is to summarize the following dialogue. Your answer should be based on the provided dialogue only.

### Dialogue:Summary this dialog:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

 ### Summary:



## Evaluation Test

In [10]:
from evaluate import load
# Load the ROUGE metric
import evaluate
rouge = evaluate.load('rouge')
def evaluate(res, ans):
    if 'llama-3' in path or 'gemma' in path:
        input_ids = tokenizer.apply_chat_template(res, add_generation_prompt=True)
        input_ids= torch.tensor(input_ids).long()
        input_ids= input_ids.unsqueeze(0)
        input_ids= input_ids.to("cuda:0")
    else:

        inputs = tokenizer(res, return_tensors="pt")
        input_ids = inputs["input_ids"].to('cuda')

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            max_new_tokens=1024,
            return_dict_in_generate=True,
            output_scores=True,
        )    
            
    s = generation_output.sequences[0]
    if "llama-3" in path or "gemma" in path:
        prediction = tokenizer.decode(s[input_ids.shape[-1]:], skip_special_tokens=True)
    else:
        output = tokenizer.decode(s)
        # print(output)
        prediction = output.split(template["response_split"])[1].strip()
        # prediction = prediction.split('</s>')[0].strip()
        prediction = prediction.split('<|im_end|>')[0].strip()
    print("Prediction:", prediction)
    print("Answer:", ans)
    results = rouge.compute(predictions=[prediction], references=[ans])
    return results['rouge1']

In [13]:
evaluate(prompt, test_data[0]['messages'][1]['content'])

Prediction: Betty's phone number is not available, so Amanda suggests asking Larry, who is a mutual friend. Amanda describes Larry as very nice, but Hannah is hesitant to reach out to him. Amanda encourages Hannah to text Larry, but Hannah is not interested. The conversation ends with a friendly goodbye.</s>
Answer: Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


0.23880597014925373